<a target="_blank" href="https://colab.research.google.com/github/tomzso/Birdclef2023-/blob/main/BirdCLEF2023_Save_raw_train_audio.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# Downloading the neccesary files from the Kaggle with kaggle.json file from one of the user
# This code cell is noly used when the file is jupyter file is opened in Google Colab
from google.colab import drive
drive.mount('/content/drive')


!pip install kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c birdclef-2023
!unzip birdclef-2023.zip


Mounted at /content/drive


Streaming output truncated to the last 5000 lines.
  inflating: train_audio/rerswa1/XC330427.ogg  
  inflating: train_audio/rerswa1/XC330925.ogg  
  inflating: train_audio/rerswa1/XC331732.ogg  
  inflating: train_audio/rerswa1/XC333007.ogg  
  inflating: train_audio/rerswa1/XC333008.ogg  
  inflating: train_audio/rerswa1/XC333855.ogg  
  inflating: train_audio/rerswa1/XC334159.ogg  
  inflating: train_audio/rerswa1/XC334160.ogg  
  inflating: train_audio/rerswa1/XC334161.ogg  
  inflating: train_audio/rerswa1/XC370598.ogg  
  inflating: train_audio/rerswa1/XC371021.ogg  
  inflating: train_audio/rerswa1/XC371367.ogg  
  inflating: train_audio/rerswa1/XC376096.ogg  
  inflating: train_audio/rerswa1/XC379520.ogg  
  inflating: train_audio/rerswa1/XC381917.ogg  
  inflating: train_audio/rerswa1/XC382091.ogg  
  inflating: train_audio/rerswa1/XC383377.ogg  
  inflating: train_audio/rerswa1/XC383401.ogg  
  inflating: train_audio/rerswa1/XC384434.ogg  
  inflating: train_audio/rerswa1/XC38

In [ ]:
#import all the necessary libraries

import os
import numpy as np
import tensorflow as tf
import librosa

In [ ]:
# Create array for X train audio data and Y train audio data

audio_data_X = []
audio_data_Y = []
audio_bird_name = []

In [ ]:
# Initialize the parameters for the iteration

dataset_path = "/content/train_audio" # The dataset path for the raw audio files in Google Colab
SAMPLE_RATE = 32000
N_MELS = 128
HOP_LENGHT = 512
duration = 5
total_sample = duration * SAMPLE_RATE
sample_rate = tf.cast(SAMPLE_RATE, dtype=tf.int64)

In [ ]:
# loop through all bird sub-folder
for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

  if dirpath is not dataset_path:
      # save bird label (i.e., sub-folder name) in the mapping
      bird_name = dirpath.split("/")[-1]
      print("\nProcessing: {}".format(bird_name))

      # process all audio files in bird sub-dir
      for filename in filenames:
        # load audio file
        file_path = os.path.join(dirpath, filename)
        signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

        # if the audio duration is lower then 5 second then it repeats the audio until it reaches 5 second
        if signal.shape[0] < total_sample:
          repeated_signal = np.tile(signal, total_sample // len(signal))
          remaining_signal = signal[:total_sample % len(signal)]
          resulting_signal = np.concatenate((repeated_signal, remaining_signal))
          signal = resulting_signal

        # if the audio duration is higher then 5 second then it chooses randomly a 5 second duration part from the original audio
        elif signal.shape[0] > total_sample:
          np.random.seed(42)
          random_number = np.random.randint(0, len(signal) - total_sample)
          signal = signal[random_number:(random_number + total_sample)]

        # Compute the Mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram( y = signal, sr=sample_rate, n_mels=N_MELS, hop_length=HOP_LENGHT)

        # Take the logarithm to create the Log-Mel Spectrogram
        log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)

        # Convert the Log-Mel Spectrogram to a NumPy array
        log_mel_spectrogram = log_mel_spectrogram.astype('float32')

        # Saving X and Y into an array
        audio_data_Y.append(np.array([i]))
        audio_data_X.append(np.array(log_mel_spectrogram))
        audio_bird_name.append(np.array([bird_name]))


Processing: brctch1

Processing: subbus1

Processing: combul2

Processing: yebsto1

Processing: gyhneg1

Processing: crefra2

Processing: brtcha1

Processing: nubwoo1

Processing: whbcro2

Processing: rostur1

Processing: yewgre1

Processing: blhher1

Processing: raybar1

Processing: wfbeat1

Processing: gryapa1

Processing: blcapa2

Processing: witswa1

Processing: golher1

Processing: fotdro5

Processing: gargan

Processing: squher1

Processing: brcwea1

Processing: gycwar3

Processing: broman1

Processing: litegr

Processing: meypar1

Processing: blhgon1

Processing: refcro1

Processing: cohmar1

Processing: hamerk1

Processing: bltori1

Processing: yetgre1

Processing: joygre1

Processing: gnhsun1

Processing: vilwea1

Processing: reftin1

Processing: somgre1

Processing: sltnig1

Processing: whhsaw1

Processing: mabeat1

Processing: yebgre1

Processing: afgfly1

Processing: ndcsun2

Processing: afpwag1

Processing: soufis1

Processing: wtbeat1

Processing: norpuf1

Processing: gy

In [ ]:
# Get and convert X to a 2D (16941,40064) from a 3D (16941,128,313)
audio_data_X_array_3D = np.array(audio_data_X)
audio_data_X_array_2D  = audio_data_X_array_3D.reshape(audio_data_X_array_3D.shape[0],-1)

# Get Y data and the bird names for each rows
audio_data_Y_np = np.array(audio_data_Y)
audio_bird_name_np = np.array(audio_bird_name)

In [ ]:
# Save the arrays into Y data and the bird names for each rows into csv files
np.savetxt("audio_data_Y.csv", audio_data_Y_np, fmt='%d', delimiter=',')
np.savetxt("audio_bird_name.csv", audio_bird_name_np, delimiter=',', fmt='%s')

In [ ]:
#Downloading the audio data into 10 equally-sized NPY files.
#The X audio data has been split into 10 individual files because having them all in one file would be too large.
#The entire dataset contains 16,941 records, with each file containing 1700 rows, resulting in the dataset being divided into 10 equally-sized individual files.
rows_per_file = 1700
for i in range(10):
    start_row = i * rows_per_file
    end_row = (i + 1) * rows_per_file

    # Extract the subset of data for this file
    subset_data = audio_data_X_array_2D[start_row:end_row]

    # Generate the filename (audio_data_X1, audio_data_X2, ...  audio_data_X9)
    filename = f'audio_data_X{i + 1}.npy'

    # Save the subset_data to the file
    np.save(filename, subset_data)


# The remaining file is split outside of the loop because its only has 16941 rows
start_row = 9 * rows_per_file
# Extract the subset of data for this file
subset_data = audio_data_X_array_2D[start_row:]

# Generate the filename (audio_data_X10)
filename = f'audio_data_X{9 + 1}.npy'

# Save the subset_data to the file
np.save(filename, subset_data)